In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                                                   logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

In [4]:
tf.summary.scalar('cost', cost)

<tf.Tensor 'cost:0' shape=() dtype=string>

In [5]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [7]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [10]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [11]:
for _ in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step),
          'Cost: %3f'  % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 3,  Cost: 1.616688
Step: 4,  Cost: 1.498244
Step: 5,  Cost: 1.394814
Step: 6,  Cost: 1.304067
Step: 7,  Cost: 1.225103
Step: 8,  Cost: 1.156908
Step: 9,  Cost: 1.098591
Step: 10,  Cost: 1.047167
Step: 11,  Cost: 1.003380
Step: 12,  Cost: 0.966646
Step: 13,  Cost: 0.935756
Step: 14,  Cost: 0.910038
Step: 15,  Cost: 0.889336
Step: 16,  Cost: 0.872261
Step: 17,  Cost: 0.858054
Step: 18,  Cost: 0.846335
Step: 19,  Cost: 0.836567
Step: 20,  Cost: 0.828206
Step: 21,  Cost: 0.821052
Step: 22,  Cost: 0.814881
Step: 23,  Cost: 0.809539
Step: 24,  Cost: 0.805299
Step: 25,  Cost: 0.801443
Step: 26,  Cost: 0.798068
Step: 27,  Cost: 0.795248
Step: 28,  Cost: 0.793056
Step: 29,  Cost: 0.791552
Step: 30,  Cost: 0.790771
Step: 31,  Cost: 0.790521
Step: 32,  Cost: 0.790759
Step: 33,  Cost: 0.791423
Step: 34,  Cost: 0.792443
Step: 35,  Cost: 0.793748
Step: 36,  Cost: 0.795260
Step: 37,  Cost: 0.796906
Step: 38,  Cost: 0.798614
Step: 39,  Cost: 0.800321
Step: 40,  Cost: 0.801967
Step: 41,  Cost: 0.

In [14]:
summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

In [16]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-102'

In [17]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('Predicition: ', sess.run(prediction, feed_dict={X: x_data}))
print('Label: ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy: %.2f' % sess.run(accuracy, feed_dict={X: x_data, Y: y_data}))

Predicition:  [0 1 2 0 0 2]
Label:  [0 1 2 0 0 2]
Accuracy: 1.00
